In [3]:
import numpy as np
import pandas as pd
import math
from sklearn.neighbors import KNeighborsRegressor

In [4]:
filename = "train.csv"
df = pd.read_csv(filename, nrows=1000000)
df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...,...
999995,2014-09-13 21:44:38.0000001,7.0,2014-09-13 21:44:38 UTC,-73.976676,40.785630,-73.959196,40.775114,1
999996,2010-09-20 14:50:37.0000002,7.3,2010-09-20 14:50:37 UTC,-73.992103,40.671385,-73.992103,40.671385,2
999997,2013-04-26 14:03:00.000000118,10.5,2013-04-26 14:03:00 UTC,-73.978423,40.751135,-73.993397,40.762197,5
999998,2011-07-08 00:29:00.00000099,6.9,2011-07-08 00:29:00 UTC,-73.980317,40.759482,-73.976832,40.743122,1


In [5]:
def distance(lat1, lat2, lon1,lon2):
    p = math.pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a))

In [6]:
def add_dist_column(df):
    df['distance'] = distance(df['pickup_longitude'],df['dropoff_longitude'] ,df['pickup_latitude'], df['dropoff_latitude'])
    return df

In [7]:
df = df.drop(df[df['fare_amount']<=0].index,axis=0)
df = df.drop(df[df['pickup_latitude']>45].index,axis=0)
df = df.drop(df[df['pickup_latitude']<40].index,axis=0)
df = df.drop(df[df['pickup_longitude']>-71].index,axis=0)
df = df.drop(df[df['pickup_longitude']<-75].index,axis=0)
df = df.drop(df[df['dropoff_latitude']>45].index,axis=0)
df = df.drop(df[df['dropoff_latitude']<40].index,axis=0)
df = df.drop(df[df['dropoff_longitude']>-71].index,axis=0)
df = df.drop(df[df['dropoff_longitude']<-80].index,axis=0)
df = df.drop(df[df['passenger_count']==0].index,axis=0)
df = df.drop('key',axis=1)
df = df.drop('pickup_datetime',axis=1)
df = add_dist_column(df)
df = df.drop(df[df['distance']==0].index,axis=0)
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
count,965198.000000,965198.000000,965198.000000,965198.000000,965198.000000,965198.000000,965198.000000
mean,11.344810,-73.975406,40.750995,-73.974539,40.751342,1.691272,1.702576
std,9.679264,0.037458,0.028850,0.037233,0.032450,1.306264,2.498776
min,0.010000,-74.968142,40.052722,-76.678448,40.041180,1.000000,0.000016
25%,6.000000,-73.992282,40.736591,-73.991578,40.735597,1.000000,0.543282
50%,8.500000,-73.982107,40.753423,-73.980623,40.753890,1.000000,0.976398
75%,12.500000,-73.968455,40.767582,-73.965478,40.768424,2.000000,1.772226
max,495.000000,-71.003917,43.183332,-71.003917,44.600000,6.000000,200.295415


In [8]:
y = df['fare_amount']
y = np.array(y,dtype=float)
X = df.drop('fare_amount',axis=1)
X = np.array(X,dtype=float)

In [9]:
regr = KNeighborsRegressor()
regr.fit(X,y)

KNeighborsRegressor()

In [10]:
df = pd.read_csv('test.csv')
df = df.drop('pickup_datetime',axis=1)
df = add_dist_column(df)
# df = df.drop(df[df['distance']==0].index,axis=0)
key = df['key']
X_test = df.drop('key',axis=1)
X_test.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
count,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000
mean,-73.974722,40.751041,-73.973657,40.751743,1.671273,1.722966
std,0.042774,0.033541,0.039072,0.035435,1.278747,2.547583
min,-74.252193,40.573143,-74.263242,40.568973,1.000000,0.000000
25%,-73.992501,40.736125,-73.991247,40.735254,1.000000,0.547182
50%,-73.982326,40.753051,-73.980015,40.754065,1.000000,0.994109
75%,-73.968013,40.767113,-73.964059,40.768757,2.000000,1.808055
max,-72.986532,41.709555,-72.990963,41.696683,6.000000,59.863893


In [11]:
y = regr.predict(X_test)
data = {'key':key,'fare_amount':y}
ans = pd.DataFrame(data)
ans.to_csv('subn.csv',index=False)
ans

/home/cmaspi/.local/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(


,key,fare_amount
0,2015-01-27 13:08:24.0000002,9.160
1,2015-01-27 13:08:24.0000003,8.760
2,2011-10-08 11:53:44.0000002,5.280
3,2012-12-01 21:12:12.0000002,7.420
4,2012-12-01 21:12:12.0000003,15.880
...,...,...
9909,2015-05-10 12:37:51.0000002,9.600
9910,2015-01-12 17:05:51.0000001,7.460
9911,2015-04-19 20:44:15.0000001,54.212
9912,2015-01-31 01:05:19.0000005,18.880
